### Luis Arce https://github.com/LuisVArce
### Tim Keriazes https://github.com/tim-keriazes
### Joshua Mayes https://github.com/MrEnigmamgine

#### Sep 22, 2022

# Red Wine From the Vihno Verde Region in Portugal - Predicting the Quality Score Using Machine Learning

#### Scenario

#### Project Description:
##### Our project examines 11 quantitative features of red/white wine data sets from the Vihno Verde region of Portugal. Using the physicochemical features/breakdown of the wine, we built a predictive machine learning model with a target variable of quality score. Our insights, discoveries, and modeling offer a distinct advantage to wine producers/stakeholders/distributors by using a wine's chemical composition and predicting its associated quality score.


#### Project Planning/Outline:
1. Intro
2. Acquire
3. Prepare/Wrangle
4. Split
5. Exploration Highlights
6. Stats Tests?
7. Scale
8. Clusters
9. Modeling
10. Conclusion
11. Next Steps

#### Initial Hypotheses
1. Sugar and alcohol content directly to correlates to wine density
2. For white wines, the higher acid content the higher quality
3. For red wines, residual sugar content lowers quality score
4. Sulfates will have negative impact on quality for both
5. High volitile acid content lowers quality for both
6. White and red wines may need predicted separately

### Target variable
#### quality
- Quality score is the median score given to associated wine based off the rankings of three industry experts

### Exploration Key Findings/Results:

#### 1. Key Finding 
#### 2. Key Finding
#### 3. Key Finding
#### 4. Key Finding
#### 5. Key Finding



# Best Model: 
### 2nd Degree Polynomial Regressor:
    - Established baseline using the mean quality score. Baseline Model had an RMSE in train of .815 and in validate set of .862. Our final model scored a RMSE of  on the test set
    
   #### Baseline:

    RMSE using Mean:
    Train/In-Sample: 0.815
    Validate/Out-of-Sample: 0.862
          
   #### RMSE for Polynomial Regressor:
    
    Test/Out-of-Sample Performance: 0.567

### Key takeaways
    -  
    -  
### Next Steps
    - 

### Imports/Dependencies

In [8]:
#imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from scipy import stats
from sklearn.cluster import KMeans

#functions

import acquire
import wrangle as wr
import warnings
warnings.filterwarnings("ignore")

#evaluate
from sklearn.metrics import mean_squared_error, r2_score, explained_variance_score
from sklearn.feature_selection import f_regression 
from statsmodels.formula.api import ols
import sklearn.preprocessing

#feature engineering
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.feature_selection import RFE
from sklearn.preprocessing import MinMaxScaler

# modeling methods
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression, LassoLars, TweedieRegressor
from sklearn.preprocessing import PolynomialFeatures

In [7]:
wr.wrangle_data("red")

FileNotFoundError: [Errno 2] No such file or directory: 'data/winequality-red.csv'

In [10]:
acquire.red_get_data()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
5,7.4,0.660,0.00,1.8,0.075,13.0,40.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1593,6.8,0.620,0.08,1.9,0.068,28.0,38.0,0.99651,3.42,0.82,9.5,6
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5
